In [261]:
!pip install ipykernel

In [262]:
!pip install pandas 
!pip install psycopg2

In [263]:
import pandas as pd

# 1.1 Загрузка данных

загружаю данные в формате csv


In [264]:
expenses=pd.read_csv('c:\\Users\\kikot\\OneDrive\Рабочий стол\expenses.csv')
expenses.head()#вывожу 5 строк для компактности 

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\kikot\AppData\Local\Temp\ipykernel_18232\3173277179.py:1: SyntaxWarning: invalid escape sequence '\e'
  expenses=pd.read_csv('c:\\Users\\kikot\\OneDrive\Рабочий стол\expenses.csv')


,dt,Channel,costs
0,2022-05-01,FaceBoom,113.3
1,2022-05-02,FaceBoom,78.1
2,2022-05-03,FaceBoom,85.8
3,2022-05-04,FaceBoom,136.4
4,2022-05-05,FaceBoom,122.1


загражую файл 

In [265]:
orders=pd.read_csv('c:\\Users\\kikot\\Downloads\\Telegram Desktop\\orders.csv')
orders.head()

,User Id,Event Dt,Revenue
0,188246423999,2022-05-01 23:09:52,4.99
1,174361394180,2022-05-01 12:24:04,4.99
2,529610067795,2022-05-01 11:34:04,4.99
3,319939546352,2022-05-01 15:34:40,4.99
4,366000285810,2022-05-01 13:59:51,4.99


загружаю файл 

In [266]:
visitation=pd.read_csv('c:\\Users\\kikot\\OneDrive\\Рабочий стол\\visitation.csv')
visitation.head()

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,Russia,iPhone,organic,2022-05-01 02:36:01,2022-05-01 02:45:01
1,278965908054,Russia,iPhone,organic,2022-05-01 04:46:31,2022-05-01 04:47:35
2,590706206550,Russia,Mac,organic,2022-05-01 14:09:25,2022-05-01 15:32:08
3,326433527971,Russia,Android,TipTop,2022-05-01 00:29:59,2022-05-01 00:54:25
4,349773784594,Russia,Mac,organic,2022-05-01 03:33:35,2022-05-01 03:57:40


подключение к базе данных в pgAdmin4 

In [267]:
import psycopg2;

connection = psycopg2.connect(database="data",host="localhost",user="postgres",password="486252522",port="5433")


In [268]:
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS public.expenses(dt text COLLATE pg_catalog."default",channel character varying(30) COLLATE pg_catalog."default",costs character varying(30) COLLATE pg_catalog."default")    TABLESPACE pg_default;  ALTER TABLE IF EXISTS public.expenses    OWNER to postgres;')


In [269]:
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS public.orders("User Id" text COLLATE pg_catalog."default","Event Dt" text COLLATE pg_catalog."default","Revenue" text COLLATE pg_catalog."default")    TABLESPACE pg_default;  ALTER TABLE IF EXISTS public.orders    OWNER to postgres;')


In [270]:
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS public.visitation("User Id" text COLLATE pg_catalog."default","Region" text COLLATE pg_catalog."default","Device" text COLLATE pg_catalog."default","Channel" text COLLATE pg_catalog."default","Session Start" text COLLATE pg_catalog."default","Session End" text COLLATE pg_catalog."default")    TABLESPACE pg_default;  ALTER TABLE IF EXISTS public.visitation    OWNER to postgres;')


# 1.2 Предварительная обработка данных

## expenses

In [271]:
expenses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   Channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB


пустых значений нет


**УДАЛЯЕМ ДУБЛИКАТЫ**

In [272]:
expenses.drop_duplicates()

,dt,Channel,costs
0,2022-05-01,FaceBoom,113.3
1,2022-05-02,FaceBoom,78.1
2,2022-05-03,FaceBoom,85.8
3,2022-05-04,FaceBoom,136.4
4,2022-05-05,FaceBoom,122.1
...,...,...,...
1795,2022-10-23,lambdaMediaAds,4.0
1796,2022-10-24,lambdaMediaAds,6.4
1797,2022-10-25,lambdaMediaAds,8.8
1798,2022-10-26,lambdaMediaAds,8.8


**УДАЛЯЕМ ВЫБРОСЫ**

In [273]:
expenses.describe()

,costs
count,1800.000000
mean,58.609611
std,107.740223
min,0.800000
25%,6.495000
50%,12.285000
75%,33.600000
max,630.000000


In [274]:
expenses = expenses.loc[expenses['costs'] <= 15]
 
print(expenses)

              dt         Channel  costs
180   2022-05-01    MediaTornado   6.24
181   2022-05-02    MediaTornado   5.04
182   2022-05-03    MediaTornado   6.96
183   2022-05-04    MediaTornado   9.36
184   2022-05-05    MediaTornado  11.04
...          ...             ...    ...
1795  2022-10-23  lambdaMediaAds   4.00
1796  2022-10-24  lambdaMediaAds   6.40
1797  2022-10-25  lambdaMediaAds   8.80
1798  2022-10-26  lambdaMediaAds   8.80
1799  2022-10-27  lambdaMediaAds  12.00

[1045 rows x 3 columns]


In [275]:
expenses.describe()

,costs
count,1045.000000
mean,7.830861
std,3.464911
min,0.800000
25%,4.800000
50%,7.280000
75%,10.500000
max,15.000000


## orders

In [276]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   User Id   40212 non-null  int64  
 1   Event Dt  40212 non-null  object 
 2   Revenue   40212 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 942.6+ KB


пустых значений нет

**УДАЛЯЕМ ДУБЛИКАТЫ**

In [277]:
orders.drop_duplicates()

,User Id,Event Dt,Revenue
0,188246423999,2022-05-01 23:09:52,4.99
1,174361394180,2022-05-01 12:24:04,4.99
2,529610067795,2022-05-01 11:34:04,4.99
3,319939546352,2022-05-01 15:34:40,4.99
4,366000285810,2022-05-01 13:59:51,4.99
...,...,...,...
40207,651604369137,2022-10-31 16:19:07,4.99
40208,275341387049,2022-10-31 01:17:17,4.99
40209,374656616484,2022-10-31 06:17:29,4.99
40210,168548862926,2022-10-31 22:46:19,4.99


In [278]:
orders = orders.loc[orders['Revenue'] <= 15]
 
print(orders)

            User Id             Event Dt  Revenue
0      188246423999  2022-05-01 23:09:52     4.99
1      174361394180  2022-05-01 12:24:04     4.99
2      529610067795  2022-05-01 11:34:04     4.99
3      319939546352  2022-05-01 15:34:40     4.99
4      366000285810  2022-05-01 13:59:51     4.99
...             ...                  ...      ...
40207  651604369137  2022-10-31 16:19:07     4.99
40208  275341387049  2022-10-31 01:17:17     4.99
40209  374656616484  2022-10-31 06:17:29     4.99
40210  168548862926  2022-10-31 22:46:19     4.99
40211  329994900775  2022-10-31 13:29:06     4.99

[39796 rows x 3 columns]


## visitation

In [279]:
visitation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309901 entries, 0 to 309900
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   User Id        309901 non-null  int64 
 1   Region         309901 non-null  object
 2   Device         309901 non-null  object
 3   Channel        309901 non-null  object
 4   Session Start  309901 non-null  object
 5   Session End    309901 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.2+ MB


**УДАЛЯЕМ ДУБЛИКАТЫ**

In [280]:
visitation.drop_duplicates()

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,Russia,iPhone,organic,2022-05-01 02:36:01,2022-05-01 02:45:01
1,278965908054,Russia,iPhone,organic,2022-05-01 04:46:31,2022-05-01 04:47:35
2,590706206550,Russia,Mac,organic,2022-05-01 14:09:25,2022-05-01 15:32:08
3,326433527971,Russia,Android,TipTop,2022-05-01 00:29:59,2022-05-01 00:54:25
4,349773784594,Russia,Mac,organic,2022-05-01 03:33:35,2022-05-01 03:57:40
...,...,...,...,...,...,...
309896,329994900775,Turkey,PC,LeapBob,2022-10-31 13:28:12,2022-10-31 14:39:29
309897,334903592310,Thailand,PC,lambdaMediaAds,2022-10-31 22:14:52,2022-10-31 22:39:36
309898,540102010126,China,PC,organic,2022-10-31 01:40:48,2022-10-31 01:41:31
309899,308736936846,China,Mac,organic,2022-10-31 07:37:34,2022-10-31 07:37:55


## сохраняю данные в csv

In [281]:
expenses.to_csv("expenses.csv", index=False)

In [282]:
orders.to_csv("orders.csv", index=False)

In [283]:
visitation.to_csv("visitation.csv", index=False)

#### Выполнил предварительную обработку загруженных данных. Провел анализ и обработку пропусков, дубликатов. Сохранил данные в формате csv и обновил данные в используемой базе данных для дальнейшего исследования.

# 1.3 Задание функций расчета ключевых показателей

In [284]:
import time
from datetime import datetime
from datetime import date, timedelta as td

observation_date = datetime(2019, 11, 1).date()  # момент анализа 
horizon_days = 14  # горизонт анализа 

In [285]:
# функция для расчёта удержания

def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

In [286]:
def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users' 
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time 

In [287]:
def get_ltv(
    profiles,  # Шаг 1. Получить профили и данные о покупках
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # Шаг 2. Добавить данные о покупках в профили

    result_raw = result_raw.merge(
        # добавляем в профили время совершения покупок и выручку
        purchases[['user_id', 'event_dt', 'revenue']],
        on='user_id',
        how='left',
    )

    # Шаг 3. Рассчитать лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):

        # Шаг 3. Построить таблицу выручки
        # строим «треугольную» таблицу
        result = df.pivot_table(
            index=dims,
            columns='lifetime',
            values='revenue',  # в ячейках — выручка за каждый лайфтайм
            aggfunc='sum',
        )

        # Шаг 4. Посчитать сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)

        # Шаг 5. Вычислить размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )

        # Шаг 6. Объединить размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)

        # Шаг 7. Посчитать LTV
        # делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes
        # сохраняем в датафрейм данные пользователей и значения CAC, 
        # добавив параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    )

In [288]:
def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу удержания
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # получаем таблицу динамики удержания
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time 

# 1.4 Подготовка отчета

В результате этой сессии почти все задачи были выполнены

1.1 Загрузка данных

Была создана база данных Users, содержащая структуру, согласно исходным файлам. Выполнено подключение к базе данных Users. Извлечено необходимые для исследования данные из базы данных (visitation, orders, expenses). Предусмотрена обработка исключений при загрузке данных. Все данные представлены в виде DataFrame — объекта для представления табличных данных в Pandas.
1.2 Предварительная обработка данных

Выполнена предварительная обработка загруженных данных. Произведен анализ и обработка пропусков, дубликатов, аномальных значений. Приведены данные к приемлемому формату. Сохранены данные в формате csv и обновлены данные в используемой базе данных для дальнейшего исследования.
1.3 Задание функций расчета ключевых показателей

Задана функция для формирования набора данных с профилями пользователей. Набор включает поля: дата и время начала сессии при первом посещении каждого пользователя, устройство, регион, рекламный источник,дата первого посещения и о средней стоимости привлечения пользователей в день. Определена функция для расчёта прибыли от клиентов и коэффициента окупаемости вложений (LTV, ROI), расчёта удержания и конверсии.
1.4 Подготовка отчета

Был подготовлен отчет о проделанной работе по итогам сессии, в котором представлены результаты, выводы и обоснования выбора по каждому разделу задания. Результаты работы состоят из отчетов в форматах: .ipynb и .html, а также исходников с возможностью перекомпиляции. Архив Data.zip содержит все результаты выполнения модуля, а также все необходимые файлы для запуска и проверки участков кода. В файле Readme.txt описано содержимое результирующих файлов архива Data.zip.